In [2]:
import os
import pandas as pd
import talib as ta
import numpy as np

# Create a list of all files in the "historical data" sub-folder
folder_path = 'historical data'
files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# Initialize a list to store each dataframe
dataframes = []

# Function to calculate the Donchian Channels
def calcDonchianChannels(data: pd.DataFrame, period: 14):
    data["upperDon"] = data["High"].rolling(period).max()
    data["lowerDon"] = data["Low"].rolling(period).min()
    data["midDon"] = (data["upperDon"] + data["lowerDon"]) / 2
    return data


# Initialize an empty list to store the dataframes
df_list = []

# Iterate over the tickers list
for file in files:
    # Fetch historical data for each ticker
    symbol = file.split('.')[0]
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep=';', header=None, names=[
                     'Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Symbol'] = symbol
    dataframes.append(df)
    df = df.reset_index()[["Date", "Open", "High", "Low", "Close"]]
    df['Date'] = pd.to_datetime(
        df['Date'], format='%Y%m%d').dt.strftime('%Y/%m/%d')
    df['linear_reg_line'] = ta.LINEARREG(df['Close'], timeperiod=20)
    lr = ta.LINEARREG(df['Close'], timeperiod=20)
    df['linear_reg_ma'] = ta.EMA(lr, timeperiod=8)
    df["Symbol"] = file
    df['ATR(5)'] = ta.ATR(df['High'], df['Low'], df['Close'], timeperiod=5)
    upper, mid, lower = ta.BBANDS(
        df['Close'], timeperiod=30, nbdevup=2, nbdevdn=2, matype=0)
    df["bb(30,2)"] = upper
    df = calcDonchianChannels(df, 14)
    df["don_14"] = df['midDon']
    df["SMA_40"] = ta.SMA(df['Close'], timeperiod=40)
    df['Signal Date'] = np.where(((df['linear_reg_line'] != df['linear_reg_ma']) &
                                  (df['linear_reg_line'].shift(1) != df['linear_reg_ma'].shift(1))) &
                                 (df['linear_reg_line'] > df['linear_reg_ma']) &
                                 (df['linear_reg_line'].shift(1) < df['linear_reg_ma'].shift(1)) &
                                 # don_14 is equal or increasing from the previous day.
                                 (df["don_14"] >= df["don_14"].shift(1)) &
                                 # Signal day and the previous day's close must close below bb(30,2).
                                 (df["Close"] < df["bb(30,2)"]) & (df["Close"].shift(1) < df["bb(30,2)"].shift(1)) &
                                 # SMA_40 is increasing from the previous day.
                                 (df["SMA_40"] > df["SMA_40"].shift(1)),
                                 df['Date'], '')
    df["Strategy"] = np.where(df["Signal Date"] != '', "DC T-Line Buy", "")
    df_list.append(df)

# Concatenate all dataframes and export to a single csv
result = pd.concat(df_list)
result.to_csv("All_Symbols_data.csv", columns=["Date", "Symbol", "Strategy", "Open", "High", "Low",
              "Close", "linear_reg_line", "linear_reg_ma", "ATR(5)", "Signal Date", "don_14", "bb(30,2)", "SMA_40"])
